In [1]:
import requests
import pandas as pd

In [12]:
def consultar(pais: str, indicador:str, pagina:int = 1, iyear:int = 1990, fyear:int = 2022):
    
    # Página de la api y path al recurso solicitado
    api_url = 'http://api.worldbank.org/v2/es'
    path = f'/country/{pais}/indicator/{indicador}'
    url = api_url + path

    # Creamos el diccionario con los parametros 
    # para el método get
    args= {
        "date":str(iyear)+":"+str(fyear),
        'page':pagina,
        "per_page":1000,
        "format":"json",
        "prefix":"Getdata",
    }
   
    return requests.get(url,params=args).json()

In [45]:
paises=pd.read_csv('../datasets/paises.csv')

seleccion = input("Ingrese un nombre de país: ")
try:
    pais = paises.loc[paises['country.value'].str.contains(seleccion, case=False),'countryiso3code'].values[0]
except:
    print("No se encontró el país, intente de nuevo")

In [46]:
indicadores = pd.read_csv('../datasets/indicadores_seleccionados.csv')

In [47]:
indicadores.head()

,Unnamed: 0,id,name,sourceNote
0,0,SP.DYN.LE00.IN,"Esperanza de vida al nacer, total (años)",La esperanza de vida al nacer indica la cantid...
1,1,SP.DYN.LE00.FE.IN,"Esperanza de vida al nacer, mujeres (años)",La esperanza de vida al nacer indica la cantid...
2,1,SP.DYN.LE00.MA.IN,"Esperanza de vida al nacer, varones (años)",La esperanza de vida al nacer indica la cantid...
3,3,3.0.Gini,Coeficiente de Gini,El coeficiente de Gini es la medida de desigua...
4,4,SE.XPD.TOTL.GD.ZS,"Gasto público en educación, total (% del PIB)",El gasto público en educación como porcentaje ...


In [39]:
tuplas = [(a,b) for a,b in zip(indicadores.id.to_list(), 
          indicadores.name.to_list())]

In [40]:
tuplas

[('SP.DYN.LE00.IN', 'Esperanza de vida al nacer, total (años)'),
 ('SP.DYN.LE00.FE.IN', 'Esperanza de vida al nacer, mujeres (años)'),
 ('SP.DYN.LE00.MA.IN', 'Esperanza de vida al nacer, varones (años)'),
 ('3.0.Gini', 'Coeficiente de Gini'),
 ('SE.XPD.TOTL.GD.ZS', 'Gasto público en educación, total (% del PIB)'),
 ('SE.COM.DURS', 'Educación obligatoria, duración (años)'),
 ('SH.XPD.TOTL.ZS', 'Gasto en salud, total (% del PIB) '),
 ('6.0.GDP_current', 'PIB ($ a precios actuales) '),
 ('NY.GDP.PCAP.CD', 'PIB per cápita (US$ a precios actuales)'),
 ('NY.GDP.MKTP.PP.CD', 'PIB, PPA ($ a precios internacionales actuales)'),
 ('IQ.SCI.OVRL', 'Nivel general de la capacidad estadística (escala 0 - 100)'),
 ('SP.POP.TOTL.FE.ZS', 'Población, mujeres (% del total)'),
 ('SP.POP.TOTL.MA.ZS', 'Población, hombres (% del total)')]

In [48]:
pais

'ABW'

In [49]:
def carga_incremental(indicador, nombre):

    consulta = consultar(pais, indicador)

    try:
        # La primera parte de la respuesta nos indica en 
        # cuantas páginas de encuentra la información
        paginas = consulta[0]["pages"]
        
    except:

        if consulta[0]['message'][0]['key'] == 'Invalid format':
            print('No hay datos para:', nombre)
        
        pass
    
    else:

        # La segunda parte nos retorna una lista de 
        # diccionarios con la información que queríamos
        datos=consulta[1]

        if paginas > 1:
            # Agregamos los valores de las otras páginas a
            # nuestra lista de diccionarios
            for pagina in range(2,paginas+1):
                datos.extend(consultar(pais, indicador, pagina)[1])
        
        # Creo el DataFrame con todos los datos
        data = pd.json_normalize(datos)

        # Me quedo con información relevante unicamente
        data = data[['countryiso3code','country.value','date','value']]

        data.rename(columns={'value':nombre},inplace=True)

        return data


In [ ]:
indicador, nombre = tuplas.pop(0)
datos = carga_incremental(indicador, nombre)

for indicador, nombre in tuplas:
    print(indicador)
    try:
        a_sumar = carga_incremental(indicador, nombre)
        a_sumar = a_sumar[['date', nombre]]
    except TypeError:
        pass
    else:
        datos = datos.merge(a_sumar, how='left', on='date')

In [44]:
datos

,countryiso3code,country.value,date,"Esperanza de vida al nacer, total (años)","Esperanza de vida al nacer, mujeres (años)","Esperanza de vida al nacer, varones (años)",Coeficiente de Gini,"Gasto público en educación, total (% del PIB)","Educación obligatoria, duración (años)",PIB ($ a precios actuales),PIB per cápita (US$ a precios actuales),"PIB, PPA ($ a precios internacionales actuales)",Nivel general de la capacidad estadística (escala 0 - 100),"Población, mujeres (% del total)","Población, hombres (% del total)"
0,ARG,Argentina,2021,NaN,NaN,NaN,NaN,NaN,14.0,NaN,10729.232578,1.082341e+12,NaN,51.202315,48.797685
1,ARG,Argentina,2020,76.813,80.122,73.405,NaN,NaN,14.0,NaN,8585.694742,9.424315e+11,78.888900,51.214146,48.785854
2,ARG,Argentina,2019,76.667,79.995,73.242,NaN,4.72417,14.0,NaN,10076.355241,1.033473e+12,80.000000,51.225802,48.774198
3,ARG,Argentina,2018,76.520,79.863,73.082,NaN,4.87774,14.0,NaN,11795.159387,1.036307e+12,81.111100,51.237348,48.762652
4,ARG,Argentina,2017,76.372,79.726,72.924,NaN,5.45432,14.0,NaN,14613.041825,1.039331e+12,85.555567,51.248990,48.751010
5,ARG,Argentina,2016,76.221,79.583,72.768,NaN,5.54549,14.0,NaN,12790.242473,8.852275e+11,88.888900,51.260974,48.739026
6,ARG,Argentina,2015,76.068,79.434,72.614,NaN,5.77611,14.0,NaN,13789.060425,8.671768e+11,95.555567,51.273483,48.726517
7,ARG,Argentina,2014,75.913,79.281,72.460,0.426660,5.36144,13.0,5.376600e+11,12334.798245,8.398967e+11,83.333333,51.286586,48.713414
8,ARG,Argentina,2013,75.756,79.124,72.306,0.422838,5.43661,13.0,6.143835e+11,13080.254732,8.496160e+11,76.666667,51.300191,48.699809
9,ARG,Argentina,2012,75.598,78.966,72.151,0.424853,5.34583,13.0,6.043785e+11,13082.664326,8.196979e+11,85.555556,51.314208,48.685792


# ETL

## Búsqueda de indicadores

Procedo a armar la lista de indicadores que estarán bajo estudio

In [54]:
trabajo = pd.DataFrame()

In [47]:
indicators = pd.read_csv('../datasets/indicadores.csv')

# Se completó los valores faltantes con 'na'
# para poder buscar sobre el dataframe
indicators.fillna('na',inplace=True)

Buscamos de forma manual el in

In [13]:
interes = input("Indicador de interés: ")
indicators.loc[indicators.name.str.contains(interes,case=False),
     ['id','name','sourceNote']]#.to_csv('temp.csv')

,id,name,sourceNote
179,3.0.Gini,Coeficiente de Gini,El coeficiente de Gini es la medida de desigua...
180,3.0.Gini_nozero,Coeficiente de Gini (Ingreso diferente de cero),El coeficiente de Gini es la medida de desigua...
192,3.1.Gini,"Gini, Rural",El coeficiente de Gini es la medida de desigua...
205,3.2.Gini,"Gini, Urbano",El coeficiente de Gini es la medida de desigua...
17088,SI.POV.GINI,Índice de Gini,El índice de Gini mide hasta qué punto la dist...


In [14]:
interes = input("Indicador de interés: ")
pick = indicators.loc[indicators.id == interes,
     ['id','name','sourceNote']]
pick

,id,name,sourceNote
179,3.0.Gini,Coeficiente de Gini,El coeficiente de Gini es la medida de desigua...


In [16]:
trabajo = pd.concat([trabajo, pick], ignore_index=True)

In [17]:
trabajo#.to_csv('../datasets/indicadores_seleccionados.csv')

,id,name,sourceNote
0,3.0.Gini,Coeficiente de Gini,El coeficiente de Gini es la medida de desigua...
